In [1]:
import numpy as np
import pywt

import seaborn as sns #绘制confusion matrix heatmap
import os

from statsmodels.tsa.ar_model import AR

import warnings

warnings.simplefilter('ignore') #忽略警告

In [ ]:
AR()

In [2]:
import scipy
import scipy.io as sio

import scipy.signal

from scipy import linalg

import pandas as pd
from sklearn.decomposition import PCA
#分类器
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split 

import xgboost
import lightgbm

#模型集成
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from mlxtend.classifier import StackingClassifier

#模型调节
from sklearn.model_selection import GridSearchCV #参数搜索
from mlxtend.feature_selection import SequentialFeatureSelector #特征选择函数 选择合适的feature

#结果可视化
from sklearn.metrics import classification_report , confusion_matrix #混淆矩阵

#相关指标
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.metrics import cohen_kappa_score

from sklearn.metrics import roc_auc_score

#二分类其多分类化
#from sklearn.multiclass import OneVsOneClassifier
#from sklearn.multiclass import OneVsRestClassifier

#from sklearn.preprocessing import StandardScaler
#from sklearn.cluster import KMeans

#距离函数 度量向量距离
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import cosine_similarity #余弦相似度

#one-hot使用
#from keras.utils import to_categorical

from sklearn.preprocessing import label_binarize

#绘图
import matplotlib.pyplot as plt

import scipy.linalg as la

import gc

%matplotlib inline

In [3]:
sample_rate = 256 #hz
origin_channel = 16 #5 channel eeg

#采集的通道
#共16 channel
#未使用的channel使用none代替
#reference:a study on performance increasing in ssvep based bci application
SAMPLE_CHANNEL = ['Pz' , 'PO3' , 'PO4' , 'O1' , 'O2' , 'Oz' , 'O9' , 'FP2' ,
                  'C4' , 'C6' , 'CP3' , 'CP1' ,
                  'CPZ' , 'CP2' , 'CP4' , 'PO8']


# 减去前多少秒数据 second
# 减去后多少秒数据 second
CLIP_FORWARD = 2
CLIP_BACKWARD = 1

# 单个小段的实验时长
trial_time = 3 #second

trial_offset = 0 #second
start_trial_time = 0 #真正的实验开始时刻
end_trial_time = 2 #真正的实验结束时刻(<trial_time)

#是否进行归一化
#reference:a study on performance increasing in ssvep based bci application
#IS_NORMALIZE = True

#是否进行滤波
#IS_FILTER = False
#EEG频率范围
#reference:a study on performance increasing in ssvep based bci application
LO_FREQ = 0.5
HI_FREQ = 40

#是否陷波
#IS_NOTCH = False
NOTCH_FREQ = 50 #陷波 工频



In [4]:
def butter_worth(data , lowcut , highcut , order=6):
    nyq = 0.5 * sample_rate
    
    lo = lowcut / nyq
    hi = highcut / nyq
    
    b,a = scipy.signal.butter(order , [lo , hi] , btype='bandpass')

    return np.array([scipy.signal.filtfilt(b , a , data[: , i]) for i in range(data.shape[1])]).reshape((-1 , origin_channel))

def load_data(filename):    
    data = sio.loadmat(file_name=filename)['data_received'] #length*16 matrix
    
    #截掉前后数据
    data = data[CLIP_FORWARD * sample_rate : - CLIP_BACKWARD * sample_rate]
    
    #滤波
    #return butter_worth(data , 0.5 , 30)

    return data

In [5]:
def separate(data , label , overlap_length):
    '''
    最长重叠长度为size长 256*3 个数据点
    '''
    train_data = []
    train_labels = []

    size = sample_rate * trial_time #一小段 256*3 个数据点
    data_length = data.shape[0]

    idx = 0

    while idx<data_length-size:
        train_data.append(data[idx : idx+size , :])
        train_labels.append(label)

        idx = idx + (size - overlap_length)

    return np.array(train_data) , np.array(train_labels)

In [6]:
def shuffle_t_v(filenames):
    np.random.shuffle(filenames)
    
    return filenames

def combine(freq):
    overlap_length = 2*256 #重叠2秒数据
    
    #保证随机性 进行置乱
    person_0_filenames = shuffle_t_v( os.listdir('data/0/%s/' % freq) )
    person_1_filenames = shuffle_t_v( os.listdir('data/1/%s/' % freq) )
    person_2_filenames = shuffle_t_v( os.listdir('data/2/%s/' % freq) )
    person_3_filenames = shuffle_t_v( os.listdir('data/3/%s/' % freq) )
    person_4_filenames = shuffle_t_v( os.listdir('data/4/%s/' % freq) )
    person_5_filenames = shuffle_t_v( os.listdir('data/5/%s/' % freq) )
    person_6_filenames = shuffle_t_v( os.listdir('data/6/%s/' % freq) )
    person_7_filenames = shuffle_t_v( os.listdir('data/7/%s/' % freq) )
    person_8_filenames = shuffle_t_v( os.listdir('data/8/%s/' % freq) )

    #打开信号文件 并 合并
    person_0 = np.concatenate([load_data('data/0/%s/' % freq + filename) for filename in person_0_filenames] , axis = 0)
    person_1 = np.concatenate([load_data('data/1/%s/' % freq + filename) for filename in person_1_filenames] , axis = 0)
    person_2 = np.concatenate([load_data('data/2/%s/' % freq + filename) for filename in person_2_filenames] , axis = 0)
    person_3 = np.concatenate([load_data('data/3/%s/' % freq + filename) for filename in person_3_filenames] , axis = 0)
    person_4 = np.concatenate([load_data('data/4/%s/' % freq + filename) for filename in person_4_filenames] , axis = 0)
    person_5 = np.concatenate([load_data('data/5/%s/' % freq + filename) for filename in person_5_filenames] , axis = 0)
    person_6 = np.concatenate([load_data('data/6/%s/' % freq + filename) for filename in person_6_filenames] , axis = 0)
    person_7 = np.concatenate([load_data('data/7/%s/' % freq + filename) for filename in person_7_filenames] , axis = 0)
    person_8 = np.concatenate([load_data('data/8/%s/' % freq + filename) for filename in person_8_filenames] , axis = 0)
        
    #============
    #训练数据分段
    person_data_0 , person_labels_0 = separate(person_0 , label = 0 , overlap_length=overlap_length)
    person_data_1 , person_labels_1 = separate(person_1 , label = 1 , overlap_length=overlap_length)
    person_data_2 , person_labels_2 = separate(person_2 , label = 2 , overlap_length=overlap_length)
    person_data_3 , person_labels_3 = separate(person_3 , label = 3 , overlap_length=overlap_length)
    person_data_4 , person_labels_4 = separate(person_4 , label = 4 , overlap_length=overlap_length)
    person_data_5 , person_labels_5 = separate(person_5 , label = 5 , overlap_length=overlap_length)
    person_data_6 , person_labels_6 = separate(person_6 , label = 6 , overlap_length=overlap_length)
    person_data_7 , person_labels_7 = separate(person_7 , label = 7 , overlap_length=overlap_length)
    person_data_8 , person_labels_8 = separate(person_8 , label = 8 , overlap_length=overlap_length)

    #合并数据
    data = np.concatenate((person_data_0 , person_data_1 , person_data_2 ,
                           person_data_3 , person_data_4 , person_data_5 ,
                           person_data_6 , person_data_7 , person_data_8 
                        ))
    
    labels = np.concatenate((person_labels_0 , person_labels_1 , person_labels_2 ,
                             person_labels_3 , person_labels_4 , person_labels_5 ,
                             person_labels_6 , person_labels_7 , person_labels_8 
                        ))
    
    return data , labels

In [7]:
def ar(data):
    X = []
    
    for i in range(16):
        X.append( AR(data[: , i]).fit().params.reshape([1 , -1]).squeeze() )
        
    return np.array(X).reshape([1 , -1]).squeeze()


def feature_extraction_ar(data):
    X = []
    
    for datum in data:
        X.append( ar(datum) )
    
    return np.array(X)

In [ ]:
acc_s = []

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data_feature = feature_extraction_ar(data) #特征提取
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = RandomForestClassifier(n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.9854111405835544
0.9893899204244032
0.9946949602122016
0.9920424403183024
0.9854111405835544
0.9893899204244032
0.9893899204244032
0.9960212201591512
0.9907161803713528
0.986737400530504
0.9827586206896551
0.9973474801061007
0.9880636604774535
0.9946949602122016
0.9840848806366048
0.9920424403183024
0.9960212201591512
0.9880636604774535
0.9907161803713528
0.9893899204244032
freq =  6 average_acc =  0.9901193633952255 var_acc =  1.6441929514736564e-05
freq =  7.5
0.9920424403183024
0.9907161803713528
0.9920424403183024
0.9827586206896551
0.993368700265252
0.9907161803713528
0.9880636604774535
0.9907161803713528
0.9880636604774535
0.9880636604774535
0.993368700265252
0.9854111405835544
0.9907161803713528
0.9893899204244032
0.9893899204244032
0.9946949602122016
0.9893899204244032
0.9920424403183024
0.9920424403183024
0.9907161803713528
freq =  7.5 average_acc =  0.990185676392573 var_acc =  7.458013494782964e-06
freq =  8.5


In [8]:
acc_s = []

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data_feature = feature_extraction_ar(data) #特征提取
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = DecisionTreeClassifier()

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.9336870026525199
0.9190981432360743
0.9310344827586207
0.9297082228116711
0.9124668435013262
0.916445623342175
0.9363395225464191
0.9217506631299734
0.9283819628647215
0.9177718832891246
0.903183023872679
0.9363395225464191
0.9217506631299734
0.9297082228116711
0.9190981432360743
0.9297082228116711
0.9230769230769231
0.9363395225464191
0.9350132625994695
0.9124668435013262
freq =  6 average_acc =  0.9246684350132627 var_acc =  8.418408628780898e-05
freq =  7.5
0.9257294429708223
0.9058355437665783
0.9496021220159151
0.9482758620689655
0.9257294429708223
0.9270557029177718
0.9230769230769231
0.9204244031830239
0.9270557029177718
0.9297082228116711
0.9297082228116711
0.9283819628647215
0.9230769230769231
0.9230769230769231
0.9257294429708223
0.9270557029177718
0.9270557029177718
0.9442970822281167
0.9217506631299734
0.9350132625994695
freq =  7.5 average_acc =  0.9283819628647214 var_acc =  9.375285831885107e-05
freq =  8.5
0.916445623342175
0.9018567639257294
0.9283819628647

In [8]:
algorithm_name = ['svm' , 'GBDT' , 'xgb' , 'lightgbm']

for idx , algorithm in enumerate([SVC , GradientBoostingClassifier , xgboost.XGBClassifier , lightgbm.LGBMClassifier]):
    
    print(algorithm_name[idx])
    
    acc_s = []

    for freq in [6 , 7.5 , 8.5 , 10]:
        print('freq = ' , freq)
        acc_s = []

        data , labels = combine(freq) #读取数据
        data_feature = feature_extraction_ar(data) #特征提取

        for t in range(20): #循环20次 取平均值 作为最终结果
            classifier = algorithm()

            X_train, X_test, y_train, y_test = train_test_split(data_feature , labels , test_size = 0.1)

            classifier.fit(X_train , y_train)

            acc = classifier.score(X_test , y_test)
            acc_s.append(acc)
            print(acc)

        print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

svm
freq =  6
0.21618037135278514
0.19363395225464192
0.1870026525198939
0.21087533156498675
0.19230769230769232
0.22148541114058357
0.21087533156498675
0.21220159151193635
0.20291777188328913
0.20954907161803712
0.1989389920424403
0.18037135278514588
0.1962864721485411
0.19230769230769232
0.19496021220159152
0.21618037135278514
0.20822281167108753
0.22281167108753316
0.17108753315649866
0.21220159151193635
freq =  6 average_acc =  0.20251989389920425 var_acc =  0.00018347768576434096
freq =  7.5
0.22679045092838196
0.17771883289124668
0.20424403183023873
0.22015915119363394
0.1976127320954907
0.20424403183023873
0.22281167108753316
0.22281167108753316
0.21220159151193635
0.20822281167108753
0.1909814323607427
0.20159151193633953
0.1883289124668435
0.20557029177718833
0.21883289124668434
0.20159151193633953
0.1976127320954907
0.20291777188328913
0.20954907161803712
0.1976127320954907
freq =  7.5 average_acc =  0.2055702917771883 var_acc =  0.00015091923534254096
freq =  8.5
0.206896551

In [8]:
acc_s = []

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data_feature = feature_extraction_ar(data) #特征提取
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = GaussianNB()

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.4588859416445623
0.3912466843501326
0.4217506631299735
0.4442970822281167
0.3925729442970822
0.4217506631299735
0.46286472148541113
0.4323607427055703
0.38726790450928383
0.4270557029177719
0.4482758620689655
0.4389920424403183
0.45225464190981435
0.4376657824933687
0.44562334217506633
0.4177718832891247
0.40583554376657827
0.4389920424403183
0.41114058355437666
0.4124668435013263
freq =  6 average_acc =  0.4274535809018568 var_acc =  0.000477928501572515
freq =  7.5
0.493368700265252
0.5384615384615384
0.5106100795755968
0.5503978779840849
0.5371352785145889
0.5212201591511937
0.5371352785145889
0.5490716180371353
0.519893899204244
0.5305039787798409
0.47877984084880637
0.5464190981432361
0.5212201591511937
0.5106100795755968
0.5305039787798409
0.5053050397877984
0.5251989389920424
0.53315649867374
0.5172413793103449
0.5437665782493368
freq =  7.5 average_acc =  0.5250000000000001 var_acc =  0.0003346035995468907
freq =  8.5
0.6724137931034483
0.6896551724137931
0.67374005

In [8]:
acc_s = []

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data_feature = feature_extraction_ar(data) #特征提取
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = KNeighborsClassifier()

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.8594164456233422
0.8236074270557029
0.8209549071618037
0.8461538461538461
0.8461538461538461
0.8395225464190982
0.8620689655172413
0.8381962864721485
0.8289124668435013
0.8196286472148541
0.8129973474801061
0.8381962864721485
0.8169761273209549
0.8421750663129973
0.850132625994695
0.8249336870026526
0.8209549071618037
0.8156498673740054
0.830238726790451
0.8249336870026526
freq =  6 average_acc =  0.8330901856763926 var_acc =  0.00020162581176255337
freq =  7.5
0.7877984084880637
0.7957559681697612
0.7970822281167109
0.7944297082228117
0.7917771883289124
0.7877984084880637
0.7758620689655172
0.7758620689655172
0.8050397877984085
0.7984084880636605
0.7970822281167109
0.7877984084880637
0.7771883289124668
0.7851458885941645
0.7931034482758621
0.7904509283819628
0.773209549071618
0.7718832891246684
0.7984084880636605
0.7891246684350133
freq =  7.5 average_acc =  0.7886604774535809 var_acc =  8.553409226829166e-05
freq =  8.5
0.7811671087533156
0.7612732095490716
0.785145888594